# <center> Data-Driven Reliability Metrics </center>
## <center> Week 3 </center>
## <center> Interactive Notebook </center>

</br>
</br>

<center>📚 Source: W3-1 Data-Driven Reliability Metrics</center>

⚠️ To make the most out of this notebook, some level of Python programing is desirable. If you've never used Python (or any other programing language) checkout resources [here](https://wiki.python.org/moin/BeginnersGuide/Programmers) to get up to speed. This is not mandatory, but will allow you to modify the code examples and complete the activities.

⚠️ This notebook has interactive elements, pleasure ensure that your notebook is set-up to use `ipywidgets` by running `jupyter nbextension enable --py widgetsnbextension --sys-prefix` in the jupyterhub terminal.

## Notebook Overview
Within this notebook, we will move from toy to real failure datasets derived from maintenance work order records. The overarching application in this notebook is to semi-automatically calculate reliability metrics from maintenance work orders in a standardised, reproducible, low-resource, manner. To achieve this, first, we will gain insight into our data by performing exploratory data analysis (EDA). Second, we will become acquianted with fundamental natural language processing (NLP) techniques, that we will use to improve our datasets quality and for identification of end-of-life events. Third, we'll use expert logic to classify and reason about end-of-life events before performing statistical data life analysis via fitting to a 2-parameter Weibull distribution.

<img src="./images/nb_3-1_excel_to_distribution.png" alt="excel to reliability metrics" width="75%;"/>

Legend
- ⚡ indicates a new concept

## Table of Contents
* [Week 2 - Recap](#week-2-recap)
* [3 - Overview of data-driven reliability metrics from maintenance work order data](#3-overview)
* [3.A - Exploratory analysis of maintenance work order data](#3-A)
    * [Activity 3.A](#3-A-activity)
* [3.B - Preparing maintenance work order data for analysis](#3-B)
    * [Activity 3.B](#3-B-activity)
* [3.C - Identification of end-of-life events within unstructured maintenance text](#3-C)
    * [3.C.1 - Basic key-word search](#3-C-1)
        * [Activity 3.C.1 - Identifying end-of-life events (EOL) via key-word search](#3-C-1-activity)
    * [3.C.2 - Expanded key-word search using word embeddings](#3-C-2)
        * [Activity 3.C.2](#3-C-2-activity)
* [3.D - Classification of identified end-of-life events](#3-D)
    * [Activity 3.D](#3-D-activity)
* [3.E - Reasoning about classified end-of-life events](#3-E)
    * [Activity 3.E](#3-E-activity)
* [3.F - Scalable statistical data life analysis](#3-F)
* [Summary](#summary)
* [Appendix](#appendix)

## Notebook Objectives
- Perform exploratory data analysis on maintenance work order data
- Load, wrangle and process maintenance work order data to extract MTBF in a standardised and reproducible way without external third-party software

## Learning Outcomes
- Understand how to load, wrangle and process maintenance work order data
- Be comfortable with exploratory data analysis
- Understand fundamental NLP concepts such as tokenization, ngrams, etc.
- Understand how to visualise information in text using visualisation packages and dimensionality reduction

## Recap from Week 2 <a class="anchor" id="week-2-recap"></a>

Provide your answers in [Menti]()
- Do you have any questions or comments from last week?

## Notebook Setup

In [1]:
# Package for ensuring code we write is formatted nicely
%load_ext nb_black

<IPython.core.display.Javascript object>

### Import Packages

Standard packages

In [2]:
import os
import itertools
import random
from collections import Counter

<IPython.core.display.Javascript object>

- [pandas](https://pandas.pydata.org/) - Package for data handling and wrangling
- [pandas_profiling](https://github.com/ydataai/pandas-profiling) - Package for generating a profile over a pandas dataframe
- [reliability](https://reliability.readthedocs.io/en/latest/index.html) - Package for performing Weibull analysis
- [plotly](https://plotly.com/graphing-libraries/) - Package for interactive visualisation

In [3]:
# Uncomment the command below if you have any issues with packages not being available in your environment
# !pip install pandas numpy pandas_profiling ipywidgets reliability plotly

<IPython.core.display.Javascript object>

In [4]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import ipywidgets as widgets
from IPython.display import display, clear_output
from reliability.Fitters import Fit_Weibull_2P
import plotly.express as px

<IPython.core.display.Javascript object>

Configuration

In [5]:
dir_path = os.path.abspath("")
pd.set_option("display.max_rows", None)

<IPython.core.display.Javascript object>

## 3 - Overview of data-driven reliability metrics from maintenance work order data <a class="anchor" id="3-overview"></a>
This section will explore identifying end-of-life events from the unstructured fields of maintenance work orders to allow us to classify them as failures, suspensions, or other.

<img src="./images/nb_3-1_flow_diagram.png"/>

This section of the notebook will take us through different stages required to identify end-of-life events from real maintenance data, and using fortuitous data allow us to calculate reliability measures at scale. Illustrated in the figure, we will explore 6 mmain steps:

- A: Maintenance work order records
- B: **Data preparation**
- C: **Identification** of end-of-life events
- D: **Classification** of end-of-life events
- E: **Reasoning** about end-of-life events, and
- F: Statistical data life **analysis**

### 3.A - Maintenance work order records <a class="anchor" id="3-A"></a>

Here we'll load a dataset containing ~40,000 maintenance records. Before we can use it in our notebook, we'll first ensure that the data is in the right format and all records have a short text description. If you have brought your own .csv dataset of work orders, feel free to place it into the `data` directory and replace the path in `data_path_large` to your files name. Please ensure that the column headers of your data are aligned with the `expected_cols` below.

In [6]:
# If you are using your own data, please change the name of the file to the name of your data.
# Otherwise, use the URL link provided in this part of the program.
data_path_large = "../data/rh_mod_v1.csv"

<IPython.core.display.Javascript object>

In [7]:
expected_cols = [
    "id",
    "description",
    "wo_order_type",
    "total_actual_costs",
    "actual_start_date",
    "actual_finish_date",
    "functional_loc_desc",
    "functional_loc",
]

<IPython.core.display.Javascript object>

In [8]:
date_cols = [
    "actual_start_date",
    "actual_finish_date",
]

df_large = pd.read_csv(
    os.path.join(dir_path, data_path_large),
    parse_dates=date_cols,
    dayfirst=True,
    encoding="ISO-8859-1",
    thousands=",",
    dtype={"description": "str", "total_actual_costs": "float"},
)

assert set(expected_cols).issubset(
    set(df_large.columns)
), "Uploaded data does not have all the expected columns"

# Lets remove any records that do not have a short text description
df_large = df_large[~df_large["description"].isna()]

# Before we continue we'll convert the functional description into a more unique name for plotting later on
df_large["object_desc"] = df_large["functional_loc_desc"].apply(
    lambda desc: " ".join(
        [word for word in desc.replace("-", " ").split(" ") if word.isalpha()]
    )
)

<IPython.core.display.Javascript object>

Lets review the dataset we have loaded

In [9]:
df_large.head(2).T

,0,1
id,1,2
description,3Y MEC SDN REPL Conveyor Belt CVR068,CVR029 Replace Conveyor Belt
basic_start_date,2/01/2017,28/10/2020
priority,High,High
maint_activity_type,PM,PDM
wo_order_type,PM02,PM01
assembly_description,Conveyor Belt CVR068,"BELT,CONVEYOR,2200XST2500,19/6,FXS/SLL"
sort_field,261-CVR068,227-CVR029
created_on,29/04/2019,6/11/2019
total_actual_costs,1.43126e+06,990589


<IPython.core.display.Javascript object>

We can quickly gain insight into our work order data by using the python package [pandas profiling](https://github.com/ydataai/pandas-profiling) which profiles the data giving us a quick overview of its contents.

In [10]:
profile = ProfileReport(
    df_large,
    title="Maintenance Work Order Data Profile",
    explorative=True,
    progress_bar=False,
)

<IPython.core.display.Javascript object>

In [11]:
profile.to_notebook_iframe()

<IPython.core.display.Javascript object>

Further information about using pandas profiling can be found [here](https://pandas-profiling.ydata.ai/docs/master/). There are a lot of articles overviewing the capabilities of this exploratory data analysis tool such as [this one](https://towardsdatascience.com/learning-pandas-profiling-fc533336edc7).

### Activity 3.A <a class="anchor" id="3-A-activity"></a>

Provide your answer in [Menti]()

- What proportion of work orders are PMs compared to corrective activities?
- Find something interesting in the data to share with the class

### 3.B - Data preparation <a class="anchor" id="3-B"></a>
<img src="./images/nb_3-1_flow_diagram-B.png"/>
</br>
Now that we have familiarity with our dataset, we need to address the elephant in the room - data quality. Our dataset has many types of data classified as `date`, `categorical`, `numerical`, and `string`. In this section, we are particularly interested in the quality of the `string` data which manifests as the short text description of our work order records. Due to the unstructured nature of text in maintenance records, they can become unwieldly due to noise.


Noise in technical user-generated text is largely produced as a result of three factors:
- 1. limited time,
- 2. space constraints, and 
- 3. technical nature of the text

As a result of these factors, technical texts such as maintenance work order short/long text will contains a lot of noise resulting from domain-specific terms, erroneous spelling, phonetic substitutions, informal grammar, etc.

It should be noted that the considerations we will make, and processes we employ, are also directly applicable to other types of technical texts such as: condition monitoring reports (vibration analysis, etc.), notification text, down time accounting text, FLAC reports, maintenance procedures, etc.

#### 3.B.1 - Fundamentals of natural language processing
Before we dive into methods to improve the quality of our text data to support downstream tasks, first, lets familiarise ourself with some fundamental concepts of natural language processing (NLP) and get some some lingo/jargon along the way. There are a few more concepts also contained in the [Appendix]() for you to review at your leisure.

If you're interested in learning more about NLP concepts in general CORE offers a [data science springboard](https://www.corehub.com.au/professional-program) that includes this and much more!

For the following exercises we will use the texts contained within our maintenance record dataset. Moreover, we'll be using the Python [Natural Language Toolkit (NLTK)](https://www.nltk.org/) for this section of the notebook, however there are many packages available for processing, cleaning, and analysing text data. 

In [12]:
# Uncomment the command below if you have any issues with packages not being available in your environment
# !pip install nltk

<IPython.core.display.Javascript object>

In [13]:
# Import required packages and functions from nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import ngrams
from nltk.text import Text

<IPython.core.display.Javascript object>

**⚡ Concept - Corpus**</br>
A corpus is a set of text documents, where a document can be a word, sentence, paragraph, report, etc.

In [14]:
mwo_corpus = df_large["description"].apply(lambda x: str(x)).tolist()
print(f"MWO corpus consists of {len(mwo_corpus)} texts")

MWO corpus consists of 45039 texts


<IPython.core.display.Javascript object>

**⚡ Concept - Tokenization**</br>
Tokenization is the process of demarcating and possibly classifying sections of a string of input characters. The resulting tokens are then passed on to some other form of processing.

In [15]:
# Load the NLTK punkt tokenizer (find out more here - https://www.nltk.org/_modules/nltk/tokenize/punkt.html)
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\61437\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<IPython.core.display.Javascript object>

In [16]:
# Lets 'tokenize' a sentence using the NLTK (Tip: include punctuation (.,) and newlines (\n) to see how it impacts tokenization)
sentence = "[CA] replace suction pipe on pump pu001 u/s"
# sentence = random.sample(mwo_corpus, 1)[0] # Uncomment to randomly sample data from the MWO corpus

tokenized_sentence = word_tokenize(sentence)
print(
    f"Input:\n{sentence}\nTokens:\n{tokenized_sentence} ({len(tokenized_sentence)} tokens)"
)

Input:
[CA] replace suction pipe on pump pu001 u/s
Tokens:
['[', 'CA', ']', 'replace', 'suction', 'pipe', 'on', 'pump', 'pu001', 'u/s'] (10 tokens)


<IPython.core.display.Javascript object>

**⚡ Concept - Vocabulary**</br>
A vocabularly is set of terms that correspond to a particular subject matter. Now that we are familiar with tokenization, we can get a feel for how big our vocabulary is. The vocabulary of maintenance texts will be quite large, even though the length of the texts are small, as references to functional locations etc will be counted as unique words.

In [17]:
# Vocabulary depends on tokenization scheme, here we'll use an off-the-shelf tokenizer. Note we are removing all the casing from our
# texts in this operation, as the MWO data uses capitalization inconsistently and erroneous. Hence, for all intensive-purposes, the computer
# thinks that ON/on/On/oN are different tokens/words, although they mean the same thing.
mwo_tokens = list(
    itertools.chain.from_iterable([word_tokenize(text.lower()) for text in mwo_corpus])
)
mwo_vocab = set(mwo_tokens)

# Lets get the vocab including casing for comparison.
mwo_tokens_cased = list(
    itertools.chain.from_iterable([word_tokenize(text) for text in mwo_corpus])
)
mwo_vocab_cased = set(mwo_tokens_cased)

print(
    f"MWO corpus has a vocabulary size of {len(mwo_vocab)} (with casuing: {len(mwo_vocab_cased)}, {(1 - len(mwo_vocab)/len(mwo_vocab_cased)) * 100:0.1f}%)"
)

MWO corpus has a vocabulary size of 7100 (with casuing: 9187, 22.7%)


<IPython.core.display.Javascript object>

In addition to the unique set of tokens (words) in our corpus (e.g. our vocabulary), it's also useful to know the word frequency distribution.

In [18]:
# Here we'll use Pythons built in 'Counter' to count the tokens in our corpus.
# This is a very handy native object for counting (see: https://docs.python.org/3/library/collections.html#collections.Counter)
mwo_token_counts = Counter(mwo_tokens)

<IPython.core.display.Javascript object>

In [19]:
# Lets look at the some statistics of our corpus including the top 10 tokens and hapaxes. A hapaxe is a token with frequency of 1.
print(f"Total tokens:\n{sum(mwo_token_counts.values())}")
print(f"Top 10 tokens:\n{mwo_token_counts.most_common(10)}")
print(f"Rarest 10 tokens:\n{mwo_token_counts.most_common()[-10:]}")


Total tokens:
238967
Top 10 tokens:
[('on', 17588), ('insp', 14398), ('1w', 10264), ('blt', 8210), ('sdn', 5737), ('lub', 4537), ('mec', 4300), ('ins', 4265), ('replace', 3893), ('off', 3420)]
Rarest 10 tokens:
[('cance', 1), ('cvr162mdm', 1), ('cvr163mdm', 1), ('p10', 1), ('program', 1), ('cvr116-pws1576', 1), ('lay', 1), ('future', 1), ('use', 1), ('only', 1)]


<IPython.core.display.Javascript object>

**⚡ Concept - Stopwords**</br>
Stopwords usually refer to the most common words in a language. For natural language processing applications, stopwords may contain less information than rarer words. For example in the text `replace the pump`, the token `the` is usually considered a stopword as it does not add any extra information (we can still understand the text as `replace pump`).

For technical language like that found in maintenance work orders, many stopwords will not be used as these texts usually lack grammar. Moreover, standardised stopword lists include words that may be crucial for tasks such as identifying failure modes in text.

In [20]:
# Load the NLTK stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\61437\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<IPython.core.display.Javascript object>

In [21]:
# Lets look at common stopwords (NLTK has these built in!)
general_stopwords = stopwords.words("english")
print(general_stopwords)
print(f"\nNumber of stopwords: {len(general_stopwords)}")

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

<IPython.core.display.Javascript object>

In [22]:
# Lets see the impact of stop word removal from technical texts
sentence_with_stopwords = "replace pump not pumping to capacity"
# sentence_with_stopwords = random.sample(mwo_corpus, 1)[
#     0
# ]  # Uncomment to randomly sample data from the MWO corpus

# Here we'll tokenize the sentence on whitespace and then remove any token that is within the stopword list
# We lowercase the tokens when we check as the stopwords have no casing
sentence_stopwords_rmv = " ".join(
    [
        token
        for token in sentence_with_stopwords.split(" ")
        if token.lower() not in general_stopwords
    ]
)

print(f"Input:\t{sentence_with_stopwords}\nOutput:\t{sentence_stopwords_rmv}")

Input:	replace pump not pumping to capacity
Output:	replace pump pumping capacity


<IPython.core.display.Javascript object>

**⚡ Concept - N-grams**</br>
Now that we are familiar with the concept of `tokens`, n-grams are simply the process of representing a set of n continguous (adjacent) tokens. Common names used under the concept of n-gram are unigram (n=1), bigram (n=2), trigram (n=3) where n is the number of contiguous tokens in the set. To make this more concrete, consider the tokens `["idler", "not", "working"]`. Here, we can extract the following sets of n-grams:
- unigram (n=1)
    - token sets: `["idler"]`, `["not"]`, `["working"]`
    - grams: idler, not, working
- bigram (n=2)
    - token sets: `["idler", "not"]`, `["not", "working"]`
    - grams: idler not, not working
- trigram (n=3):
    - token sets: `["idler", "not", "working"]`
    - grams: idler not working
    
As we can see, chunks of grams can represent different concepts, in this example the bigram "not working" has important meaning in maintenance applications.

In [23]:
# Lets build some ngrams using NLTK (note: NLTK expects the sentence to be tokenized)
sentence_for_ngrams = "idler not working"
# sentence_for_ngrams = random.sample(mwo_corpus, 1)[0] # Uncomment to randomly sample data from the MWO corpus

# Tokenize sentence (we'll use the NLTK punkt tokenizer here)
sentence_for_ngrams_tokenized = word_tokenize(sentence_for_ngrams)

n_values = [1, 2, 3, 4]

for n in n_values:
    # The ngram function returns a `zip` object
    grams = [" ".join(gram) for gram in ngrams(sentence_for_ngrams_tokenized, n)]
    print(f"{n}-grams: {grams}\n")

1-grams: ['idler', 'not', 'working']

2-grams: ['idler not', 'not working']

3-grams: ['idler not working']

4-grams: []



<IPython.core.display.Javascript object>

⚡ **Concept - Phrases/Chunking**</br>
As seen above, tokens within a text can form many different sized ngrams ('not', 'working' vs 'not working'). How do we know which ngrams should be formed between which tokens? To do this we can "chunk" or "phrase" our text. Here we will train a phrasing algorithm that will learn to detect common phrases aka multi-word expressions automatically from our corpus.

This is an important tool in our NLP toolkit as it enables us to treat words as chunks of meaning rather than just a bag of individual words. 

In [24]:
# Uncomment the command below if you have any issues with packages not being available in your environment
# !pip install gensim

<IPython.core.display.Javascript object>

In [25]:
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS

<IPython.core.display.Javascript object>

In [26]:
print(ENGLISH_CONNECTOR_WORDS)

frozenset({'for', 'at', 'the', 'an', 'of', 'and', 'a', 'in', 'on', 'or', 'from', 'by', 'without', 'with', 'to'})


<IPython.core.display.Javascript object>

In [27]:
# Here we'll build a phrase model to detect common phrases
# Find out more: https://radimrehurek.com/gensim/models/phrases.html

# We need to feed in our corpus after it has been tokenized
tokenized_mwo_corpus_for_embedding = [
    word_tokenize(text.lower()) for text in mwo_corpus
]

# Now we'll train our phrasing model but only account for words with a minimum frequency of 5
# (you can make this lower such as 1 to account for hapaxes)
phrase_model = Phrases(
    tokenized_mwo_corpus_for_embedding,
    min_count=5,
    threshold=10,
    connector_words=ENGLISH_CONNECTOR_WORDS,
)

<IPython.core.display.Javascript object>

In [28]:
# Lets try the phrasing model out
text_to_phrase = "change out pump impeller - not working"
text_to_phrase_tokenized = word_tokenize(text_to_phrase)

# Now we'll pass it into our phrasing model
phrase_model[text_to_phrase_tokenized]

['change_out', 'pump', 'impeller', '-', 'not_working']

<IPython.core.display.Javascript object>

**⚡ Concept - Word Representation and Similarities**</br>
The last concept we'll review is that of similarity between words. To make this concrete, lets think about the words `oil`, `fuel`, `coolant` and `water`. From our background knowledge, we know these share similar meaning, i.e. they are all fluids and they may be found in industrial contexts. However, how do we make machines gain this intution? One answer is `word embeddings`.

Like the saying "you can tell a word by the company that it keeps", word embeddings are numerical representations that allows word with similar meaning to have similar values. Given a large enough corpus, these word embeddings can be learnt. 

All state-of-the-art NLP use embeddings in one form or another, so an intuition toward them is important. However, diving into the details of embeddings is out of the scope of this notebook, but if you're interested check out [here](https://machinelearningmastery.com/what-are-word-embeddings/) for further information.

<img src="./images/nb_3-1_embeddings.png" alt="word embeddings" width="75%"/>

Here we'll load more packages
- [gensim](https://radimrehurek.com/gensim/) - package used to learn word embeddings from text
- [sklearn](https://scikit-learn.org/) - package for machine learning that we'll use for dimensionality reduction
- [numpy](https://numpy.org/) - package for dealing with arrays and numerical information
- [plotly](https://plotly.com/) - package for interactive visualisation

In [29]:
# Uncomment the command below if you have any issues with packages not being available in your environment
# !pip install gensim sklearn

<IPython.core.display.Javascript object>

In [30]:
from gensim.models import Word2Vec
from gensim.models import phrases
from sklearn.decomposition import PCA

<IPython.core.display.Javascript object>

In [31]:
class EmbeddingTrainer:
    def __init__(self):
        self.min_token_count = 2
        self.window_size = 3
        self.model_size = 300

    def train_model(self, docs, iterations: int = 50):
        w2v_model = Word2Vec(
            sentences=docs,
            vector_size=self.model_size,
            min_count=self.min_token_count,
            window=self.window_size,
            epochs=iterations,
        )
        print(f"Model summary:\n {w2v_model}")
        return w2v_model

<IPython.core.display.Javascript object>

Here we'll train a word embedding model called Word2Vec ([find out more here](https://radimrehurek.com/gensim/models/word2vec.html)). Please note that we will be doing this on the original corpus without any normalisation or cleaning.

In [32]:
# First lets tokenize our corpus using the punkt tokenizer from NLTK; we'll also lower case everything to
# improve the support of each word. However, this is not prescriptive.
tokenized_mwo_corpus_for_embedding = [
    word_tokenize(text.lower()) for text in mwo_corpus
]

<IPython.core.display.Javascript object>

In [33]:
# Again, we'll build a phrase model to detect common phrases
phrase_model = phrases.Phrases(
    tokenized_mwo_corpus_for_embedding,
    min_count=5,
    threshold=10,
    connector_words=phrases.ENGLISH_CONNECTOR_WORDS,
)

<IPython.core.display.Javascript object>

In [34]:
# Lets learn word embeddings from the text in our corpus
# Here we will pass our tokenized corpus into our pretrained phrasing model

w2v_model = Word2Vec(
    sentences=[phrase_model[t] for t in tokenized_mwo_corpus_for_embedding],
    vector_size=300,
    min_count=5,
    window=3,
    epochs=50,
)

<IPython.core.display.Javascript object>

In [35]:
# Vocabulary size of the model we created
print(f"Size of word embedding vocabulary: {len(w2v_model.wv.index_to_key)}")

Size of word embedding vocabulary: 2070


<IPython.core.display.Javascript object>

Now that we have a numerical representation of words in our corpus, lets inspect some word similarities. Try words such as `seized`, `replaced`, etc.

In [36]:
def get_similar_words(model=w2v_model, word: str = ""):
    try:
        if word in w2v_model.wv.index_to_key:
            sim_words = model.wv.most_similar(word)
            print(
                "\n".join(
                    [
                        f"{word} ({similarity*100:0.1f}%)"
                        for word, similarity in sim_words
                    ]
                )
            )
        else:
            print("Word not in vocabulary - try again!")
    except Exception as e:
        print(f"Failed due to {e}")

<IPython.core.display.Javascript object>

In [37]:
# Caution: We need to ensure that we use words that exist in our vocabulary!
base_word = "seized"

get_similar_words(word=base_word)

siezed (78.7%)
pizza_cutter (70.4%)
collapsed (70.3%)
collasped (69.9%)
colapsed (69.4%)
failed (67.9%)
flat (67.5%)
noisy (67.4%)
u/s (64.9%)
callapsed (63.5%)


<IPython.core.display.Javascript object>

⚠️ An important caveat for using embeddings in technical languages is the type of embedding may be restricted on what can be represented depending on the way the embeddings were learnt. In this notebook, we are using `word` embeddings, hence the embeddings are built off of the words in our vocabulary (over the minimum threshold we specified). Hence, the embedding model will NOT represent words that are hapaxes if we set our threshold to greater than 1, it will be considered OUT OF VOCABULARY.

This is an important area for performing NLP on technical languages when using off-the-shelf embedding models (which is the default approach in NLP for most domains). First, important domain specific words may not be present (e.g. `primary_scraper` is not a common word in the English language and has specific meaning in technical contexts). Second, even if the embedding model can take the word, it doesn't mean that the context it creates from it, is correct.

This isn't to say that off-the-shelf embeddings cannot be used, as they still have great utility (being trained on billions of words), they just need to be used with an understanding of their potential limitations in technical settings.

Note: Not all embeddings are WORD embeddings, there are also sub-word embeddings, character embeddings, document embeddings, etc.

Lets quickly quantify this issue that we've discussed above by comparing our domain-specific embeddings to those from a general domain.

In [38]:
# Lets quickly look at this in action for word embeddings.
# Source: https://radimrehurek.com/gensim/auto_examples/howtos/run_downloader_api.html#how-to-download-pre-trained-models-and-corpora
import gensim.downloader as api

corpus = api.load("text8")
general_embedding_model = Word2Vec(corpus)

<IPython.core.display.Javascript object>

In [39]:
# Get similar words to our base word using the general model
get_similar_words(model=general_embedding_model, word=base_word)

attacked (76.5%)
surrendered (75.9%)
secured (74.5%)
captured (72.7%)
ousted (72.3%)
besieged (72.0%)
seizing (71.5%)
recaptured (70.7%)
succeeded (70.6%)
invaded (70.4%)


<IPython.core.display.Javascript object>

In [41]:
# lets try a domain-specific abbreviation, 'c/o' and see what happens.
get_similar_words(model=general_embedding_model, word="c/o")

Failed due to "Key 'c/o' not present in vocabulary"


<IPython.core.display.Javascript object>

In [40]:
# Check how similar words are
word_a = "seized"
word_b = "failed"
print(f"Similarity between {word_a} and {word_b}")
print(f"Domain-specific model: {w2v_model.wv.similarity(word_a, word_b) * 100:0.1f}%")
print(
    f"General model: {general_embedding_model.wv.similarity(word_a, word_b) * 100:0.1f}%"
)

Similarity between seized and failed
Domain-specific model: 67.9%
General model: 34.6%


<IPython.core.display.Javascript object>

Before we move onto the next section, lets visualise the embeddings we have learnt from our MWOs using the package `plotly`. Recall that the embeddings we created are of the size 300, hence, each word has 300 numerical values. To make this interpretable, we need to reduce the dimensionality to 2 so we can visualise it on a plot. To do this, we'll use a dimensionality reduction technique called [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) (note there are many other techniques out there).

The main reason for visualising embeddings is to identify semantic and syntactic trends in the data. For instance, words like 'replace', 'replacement', 'change out', 'repair' lie close together due to their semantics (they are activities), and 'frame' and 'frames' are close together due to syntax (plurality).

In [42]:
# First we need to build a vector of words and their embeddings
X = np.array([w2v_model.wv[word] for word in w2v_model.wv.index_to_key])
print(X.shape)

(2070, 300)


<IPython.core.display.Javascript object>

In [43]:
# Lets reduce the dimensionality of our embedding array
pca = PCA(n_components=2)
X_pca = pca.fit(X).transform(X)

<IPython.core.display.Javascript object>

In [67]:
# Lets look at the representations learnt for individual words from our MWO corpus
token_to_show = "bearing"
token_to_show_sim_tokens = [
    token_sim[0] for token_sim in w2v_model.wv.most_similar(token_to_show, topn=100)
]

df_token_sims = pd.DataFrame(
    {
        "x": X_pca[:, 0],
        "y": X_pca[:, 1],
        "word": w2v_model.wv.index_to_key,
        "label": [
            "neighbour" if token != token_to_show else "token"
            for token in w2v_model.wv.index_to_key
        ],
        "size": [
            1 if token != token_to_show else 10 for token in w2v_model.wv.index_to_key
        ],
    },
    columns=["x", "y", "word", "label", "size"],
)

# Filter for neighbouring similar tokens
df_token_sims = df_token_sims[
    df_token_sims["word"].isin([token_to_show] + token_to_show_sim_tokens)
]

fig = px.scatter(
    df_token_sims,
    x="x",
    y="y",
    text="word",
    title=f"Maintenance Work Order PCA - Token: {token_to_show}",
    color="label",
    size="size",
)
fig.update_traces(textposition="top center")
fig.update_layout(xaxis_title="Dimension 1", yaxis_title="Dimension 2")
fig.show()

<IPython.core.display.Javascript object>

In [59]:
## Uncomment the following code to visualise the entire embedding space
## Visualise the 2-dimensional data and explore
## Note: the data will become less 'messy' if you were to subset the dataset on particular assets of interest
# tokens_to_show = 500
# df_embed_original = pd.DataFrame(
#     {
#         "x": X_pca[:, 0],
#         "y": X_pca[:, 1],
#         "word": w2v_model.wv.index_to_key,
#         "label": np.array(["original"] * len(X_pca)),
#     },
#     columns=["x", "y", "word", "label"],
# )

# fig = px.scatter(
#     df_embed_original.iloc[:tokens_to_show],
#     x="x",
#     y="y",
#     text="word",
#     title="Principle Component Analysis - Embedded Words from maintenance work orders",
# )
# fig.update_traces(textposition="top center")
# fig.update_layout(xaxis_title="Dimension 1", yaxis_title="Dimension 2")
# fig.show()

<IPython.core.display.Javascript object>

Being able to visualise our technical text can provide us with immediate insight into the way information is stated in text, but what else can we do with these numerical representations? We could:
- Find activities performed in response to an end-of-life event of interest
- Aggregate the word level embeddings for entire work orders and have a document embedding we could use to find work orders that share similar meaning.

## Activity 3.B.1

Post your answers on [Menti]()
- What are your thoughts on the fundamentals of NLP? Is there anything surprising that you have learnt?
- How do you deal with noisy text in your work?
- How would you tackle fixing this text - "re*pl;ace the ## 1 p/p   impeller!! @ the pump stn"?

### 3.B.1 - Takeaways
- Tokenization is a crucial part of natural language processing
- The cleanliness of data can impact on the ability to tokenize
- Using text preprocessing blindly can accidentally remove meaning from texts e.g. removing stopwords that have technical important
- Embeddings are a powerful technique for numerically representing meaning in text 

#### 3.B.2 - Text Cleaning
There are various methods that can be adopted to improve the quality of text data such as using off-the-shelf tools, dictionaries, learning algorithms, etc.

For technical language, which maintenance records are composed of, off-the-shelf tools do not work very effectively due to the specific language used. Dictionaries are a popular approach due to their simplicity, for example a dictionary could be of the format:

```
rp : replace
rple : replace
replc : replace
```

However, the use of such dictionaries is challenging as they are most useful in `find-and-replace` strategies. For example, `rple pump impeller` we could find that the word `rple` exists in our dictionary and normalise it to `replace`. This would result in improved coverage of any techniques we were to apply to our data, however in instances where words are ambiguous and change meaning based on their context, this becomes difficult. Consider the two texts `replace a/c cable` and `replace a/c vents`. The former may be an abbreviation for `a/c -> alternating current` and the latter `a/c -> air conditioner`, hence, by doing a `find-and-replace` strategy we may erroneous normalise our data.

Here we'll develop a function to perform some basic text cleaning. In this notebook, our main focus is on identifying words and phrases that indicate end-of-life, hence identifiers, numbes, etc, can be removed as they increase our vocabulary unncessarily. The steps we'll perform here, including:
- Removing special characters
    - `**c/out gearbox` → `c/out gearbox`
- Removing superfluous whitespace
    - `idler    bearing replacement` → `idler bearing replacement`
- Removing stopwords (cautiously)
    - `replace the bearing` → `replace bearing`
- Replacing known noisy words/abbreviations with a controlled dictionary
    - `c/out bearing` → `change out bearing`

Lets write functions to perform each of these cleaning stages. We'll use an exemplar text that has the form of `re*pl;ace the ## 1 p/p   impeller!! @ the pump stn` and our goal is to normalise it deterministacally to **`replace the number 1 pump impeller at the pump station`**.

Note: What we're creating here are called [lambda functions](https://www.w3schools.com/python/python_lambda.asp) in python.

Here we'll use the a package called `regex` which is for [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) in Python. Note there is a native package in python called `re` but it can be limited in its flexibility. There are many tools to help you develop regular expressions online such as [regexr](https://regexr.com/).

In [70]:
# Uncomment the command below if you have any issues with packages not being available in your environment
# !pip install regex

<IPython.core.display.Javascript object>

In [71]:
import regex

<IPython.core.display.Javascript object>

In [72]:
test_text = "re*pl;ace the ## 1 p/p   impeller!! @ the pump stn & clean/inspect o-rings"

<IPython.core.display.Javascript object>

In [73]:
# Remove all characters except for alphanumerical and reserved special characters e.g. @, -, #, /, and .
# Note: We're using regular expressions here, so some characters need to be 'escaped' as they are special 'metacharacters'
# Refer to this for further information: https://www3.ntu.edu.sg/home/ehchua/programming/howto/Regexe.html

chars_to_keep = "&\.\#\@\/-"

fnc_rmv_chars = lambda text: regex.sub(rf"[^a-zA-Z0-9 {chars_to_keep}]", "", text)
fnc_rmv_chars(test_text)

'replace the ## 1 p/p   impeller @ the pump stn & clean/inspect o-rings'

<IPython.core.display.Javascript object>

In [74]:
# Remove duplicate reserved special characters (like ##, @@@, etc.)
# This way if we want to normalise # to number and we have ## we wont' get numbernumber

fnc_rmv_dupe_chars = lambda text: regex.sub(rf"([{chars_to_keep}])\1+", r"\1", text)

fnc_rmv_dupe_chars(test_text)

're*pl;ace the # 1 p/p   impeller!! @ the pump stn & clean/inspect o-rings'

<IPython.core.display.Javascript object>

In [75]:
# Break any erroneous hyphenated or compound abbreviations
# We want to keep things like 'o-ring' and 'c/o' but fix things like 'CV001-replace' → 'CV001 - replace'
# and 'change-out' → 'change out', 'Plate/Lower' → 'plate / lower', 'inspect/replace' → 'inspect / replace'

test_text_2 = "CVR231-Replace brake switch and change/out o-ring"

fnc_fix_compound_hyphen = lambda text: regex.sub(r"(?<=\w{3,})-(?=\w{3,})", " - ", text)
fnc_fix_compound_fwd_slash = lambda text: regex.sub(
    r"(?<=\w{3,})\/(?=\w{3,})", " / ", text
)
fnc_fix_compound_chars = lambda text: fnc_fix_compound_fwd_slash(
    fnc_fix_compound_hyphen(text)
)

fnc_fix_compound_chars(test_text_2)

'CVR231 - Replace brake switch and change / out o-ring'

<IPython.core.display.Javascript object>

In [76]:
# Removing superfluous whitespace (e.g. more than 1 whitespace between words)
# Doing this means we do not have tokens that are whitespace e.g. 'replace   engine' → ['replace', '', '', 'engine']
# We also 'strip' leading and ending whitespace e.g. ' replace engine ' → 'replace engine'

fnc_rmv_whitespace = lambda text: regex.sub(r" {2,}", " ", text).strip()
fnc_rmv_whitespace(test_text)

're*pl;ace the ## 1 p/p impeller!! @ the pump stn & clean/inspect o-rings'

<IPython.core.display.Javascript object>

In [77]:
# Removing stopwords (cautiously)
# Note: stopwords are unigrams, so we can simply split our text on whitespace and check whether each token
# is in a stopword list; if it is, skip it, otherwise keep it.

# Recall that some stopwords are meaningful for us, hence we will filter the list down first.

stopwords_to_keep = [
    "on",
    "off",
    "over",
    "under",
    "no",
    "not",
    "don",
    "don't",
    "aren",
    "aren't",
    "no",
    "not",
    "didn't",
    "doesn",
    "doesn't",
    "hadn",
    "hadn't",
    "hasn",
    "hasn't",
    "haven",
    "haven't",
    "isn",
    "isn't",
    "won",
    "won't",
    "wouldn",
    "wouldn't",
]
filtered_stopwords = [
    word for word in stopwords.words("english") if word not in stopwords_to_keep
]

fnc_rmv_stopwords = lambda text: " ".join(
    [token for token in text.split(" ") if token not in filtered_stopwords]
)

fnc_rmv_stopwords(test_text)

're*pl;ace ## 1 p/p   impeller!! @ pump stn & clean/inspect o-rings'

<IPython.core.display.Javascript object>

In [78]:
# Normalising noisy words with a controlled dictionary

word_normalisation_dictionary = {
    "#": "number",
    "@": "at",
    "u/s": "unserviceable",
    "changeout": "change out",
    "c/o": "change out",
    "c/out": "change out",
    "rplc": "replace",
    "p/p": "pump",
    "stn": "station",
    "repl": "replace",
    "&": "and",
    "rpl": "replace",
}

# Note this could be performed entirely with more complex regular expressions
def dictionary_normalisation(text: str, norm_dict: dict) -> str:
    for noisy_word, clean_word in norm_dict.items():
        if noisy_word in chars_to_keep:
            text = text.replace(noisy_word, f" {clean_word} ")
        text = regex.sub(rf"(?<!-)\b{noisy_word}\b(?!-)", f" {clean_word} ", text)

    return text


# test on test_text
dictionary_normalisation(text=test_text, norm_dict=word_normalisation_dictionary)

're*pl;ace the  number  number  1  pump    impeller!!  at  the pump  station   and  clean/inspect o-rings'

<IPython.core.display.Javascript object>

In [79]:
# Putting them all together. Note the order of these operations is important.
# We don't want to try and remove stopwords or normalise words when they have special characters, or extra whitespace etc.


def text_cleaner(text: str, norm_dict: dict) -> str:
    """Cleans and normalises a given text. Steps performed include: remove casing,
    removing special characters, removing duplicate non-alphanumerical characters, fix erroneous compound chars,
    removing stopwords, normalising terms using a dictionary, and remvoing superfluous whitespace"""
    text = text.lower()
    text = fnc_rmv_chars(text)
    text = fnc_rmv_dupe_chars(text)
    text = fnc_fix_compound_chars(text)
    text = fnc_rmv_stopwords(text)
    text = dictionary_normalisation(text, norm_dict)
    text = fnc_rmv_whitespace(text)

    return text

<IPython.core.display.Javascript object>

In [80]:
# Lets check whether we have achieved our goal of normalising the starting text
cleaned_text = text_cleaner(text=test_text, norm_dict=word_normalisation_dictionary)
print(f"Input: {test_text}\nOutput: {cleaned_text}")

Input: re*pl;ace the ## 1 p/p   impeller!! @ the pump stn & clean/inspect o-rings
Output: replace number 1 pump impeller at pump station and clean / inspect o-rings


<IPython.core.display.Javascript object>

In [81]:
# Lets quickly look at the impact of our cleaning on a set of corrective work orders
corrective_texts_to_clean = (
    df_large[df_large["wo_order_type"] == "PM01"]["description"].sample(n=10).tolist()
)

corrective_text_pairs = [
    (
        original_text,
        text_cleaner(text=original_text, norm_dict=word_normalisation_dictionary),
    )
    for original_text in corrective_texts_to_clean
]

for pair in corrective_text_pairs:
    original_text, cleaned_text = pair
    print(f"{original_text} → {cleaned_text}\n")

CVR116 Trip → cvr116 trip

CVR302 repair to V plough → cvr302 repair v plough

Installation of Perma Lubes CVR116 → installation perma lubes cvr116

CVR042 Replace collapsed return roller → cvr042 replace collapsed return roller

Replace anybus module CVR063-AYS001 → replace anybus module cvr063 - ays001

CVR065 - Gearbox Oil Over Full → cvr065 - gearbox oil over full

CVR068 Return Roller Noisy → cvr068 return roller noisy

Replace Faulty lighting along CVR121 → replace faulty lighting along cvr121

Belt Rip on CVR064 wont reset → belt rip on cvr064 wont reset

CVR069 change rollers non impact → cvr069 change rollers non impact



<IPython.core.display.Javascript object>

The steps we've seen are typical for preprocessing noisy natural language texts, however newer technology has currently started to focus on this problem using deep learning algorithms. Similar to technologies such as the spell checker on your mobile phone, and services such as Grammarly, these technologies aim to fix noisy text without writing heuristics as we've done here.

However, the technical text which you deal with is not amenable to these types of services as the language used is too domain-specific. We will not go into the details of it in this notebook, but research in the [UWA NLP-TLP group](https://nlp-tlp.org/) is developing translation models to go from noisy technical text to clean technical text without any rules. Just like English to French translation, the group treats noisy to clean text in a similar way, going from:

```
Input: re*pl;ace the ## 1 p/p   impeller!! @ the pump stn
Output: replace number 1 pump impeller at pump station
```

Before we move onto the next section, lets quickly overlay our data after its cleaned on the embeddings we made previously.

In [82]:
# Create dimensionality reduced embedding data for cleaned dataset
cleaned_tokenized_mwo_corpus_for_embedding = [
    word_tokenize(text_cleaner(text=text, norm_dict=word_normalisation_dictionary))
    for text in mwo_corpus
]
cleaned_phrase_model = phrases.Phrases(
    cleaned_tokenized_mwo_corpus_for_embedding,
    min_count=5,
    threshold=10,
    connector_words=phrases.ENGLISH_CONNECTOR_WORDS,
)
cleaned_w2v_model = Word2Vec(
    sentences=[
        cleaned_phrase_model[t] for t in cleaned_tokenized_mwo_corpus_for_embedding
    ],
    vector_size=300,
    min_count=5,
    window=3,
    epochs=50,
)
X_cleaned = np.array(
    [cleaned_w2v_model.wv[word] for word in cleaned_w2v_model.wv.index_to_key]
)
pca_cleaned = PCA(n_components=2)
X_pca_cleaned = pca.fit(X_cleaned).transform(X_cleaned)

<IPython.core.display.Javascript object>

In [101]:
def embedding_comparison(
    model_a, model_b, X_pca_a, X_pca_b, token_to_show: str, topn: int = 100
):
    """
    Compares the dimensionality reduced representation of embedding models on a given token.

    a denotes baseline model
    b denotes cleaned model
    """

    dfs = []
    for name, model, X_pca in [("", model_a, X_pca_a), ("-cleaned", model_b, X_pca_b)]:
        token_to_show_sim_tokens = [
            token_sim[0]
            for token_sim in model.wv.most_similar(token_to_show, topn=topn)
        ]

        df_token_sims = pd.DataFrame(
            {
                "x": X_pca[:, 0],
                "y": X_pca[:, 1],
                "word": model.wv.index_to_key,
                "label": [
                    f"neighbour{name}" if token != token_to_show else f"token{name}"
                    for token in model.wv.index_to_key
                ],
                "size": [
                    1 if token != token_to_show else 10
                    for token in model.wv.index_to_key
                ],
            },
            columns=["x", "y", "word", "label", "size"],
        )

        # Filter for neighbouring similar tokens
        df_token_sims = df_token_sims[
            df_token_sims["word"].isin([token_to_show] + token_to_show_sim_tokens)
        ]

        dfs.append(df_token_sims)

    return pd.concat(dfs)

<IPython.core.display.Javascript object>

In [102]:
# Lets look at the representations learnt for individual words from our MWO corpus
comparison_token_to_show = "bearing"

df_embed_comparison = embedding_comparison(
    model_a=w2v_model,
    model_b=cleaned_w2v_model,
    X_pca_a=X_pca,
    X_pca_b=X_pca_cleaned,
    token_to_show=comparison_token_to_show,
    topn=25,
)

fig = px.scatter(
    df_embed_comparison,
    x="x",
    y="y",
    text="word",
    title=f"Maintenance Work Order PCA - Token: {comparison_token_to_show}",
    color="label",
    size="size",
)
fig.update_traces(textposition="top center")
fig.update_layout(xaxis_title="Dimension 1", yaxis_title="Dimension 2")
fig.show()

<IPython.core.display.Javascript object>

Because embedings are learnt based on co-occurrence, e.g. similiar words are used in similar contexts, we see that the cleaning effort makes a difference on embedding similarities by improving the numerical representations learnt. In the next sections of this notebook, and next week, we'll further see why this stage is important for natural language processing on technical texts.

A natural question that occurs when using dictionary based lexical normalsiation is where do the dictionaries come from? There are two methods - 1. you can curate them manually from your own knowledge and available resources, or 2. use available software such as [LexiClean](https://lexiclean.nlp-tlp.org) (developed by the UWA NLP-TLP group) which helps you quickly normalise texts and build replacement dictionaries.

### Activity 3.B.2 Part A <a class="anchor" id="3-B-activity"></a>

Given the text below:
- Tokenize it using the NLTK tokenizer, and
- Extract all of the phrases.

In [104]:
# Activity 3.B.2 Part A
a3b2a_text = "[PM] change out idler bearing."

<IPython.core.display.Javascript object>

Uncomment and run the following cell to see the answer to this activity (after you try it!).

In [105]:
# %load ./solutions/W3/tokenization_phrasing.py

<IPython.core.display.Javascript object>

### Activity 3.B.2 Part B <a class="anchor" id="3-B-2-activity"></a>

Given the following MWO corpus, first clean the texts and then find the total number of tokens and the top 10 most frequent tokens.

Optional:
- Top 10 rarest tokens

In [106]:
# Corpus for getting intuition for fundemantals of NLP
# this will include noisy data to ensure that we can highlight the rarity of words, etc.
random.seed(1234)
a3b2b_corpus = random.sample(mwo_corpus, 25)
print("\n".join(a3b2b_corpus))

161/162 Transfer repairs parent metal
8W ELE ON INSP Conveyor CVR029
12W MEC SDN REPL Damaged Idlers CVR029
4W MEC ON INSP Transfer Chute CVR323
4W LUB ON INSP CVR003 SSC008
8W MEC SDN RPL Idlers CVR301
1W BLT ON INSP CVR161
48W OFF Replace Sec Scraper Tips CVR062
CVR005 Scope CVR005 Chute Mods Xfer
2W LUB ON INSP Conveyor CVR063
4W MEC ON INSP Conveyor CVR068
CVR023 BRK001 Replace Caliper/Brake Pad
CVR061 Replace Idlers
Modify Belt Reeler Kickrails
6M MEC SDN CPMS T/U Winch CVR123
1W BLT ON INSP CVR165
1W LUB ON INSP CVR122
4W CMN ON INSP CVR163
CVR115 Standard belt replacement
Supply Oil Cooler Motor CVR122
13W MEC SDN INS Brake CVR041
Impact idler frame tab broken off.
CVR065 Scope Boom Spray Water Valve
CVR061 Replace idlers and rollers
CVR241 RPL Sec & Tert Scrapers


<IPython.core.display.Javascript object>

In [107]:
# Use this area for activity 3.B


<IPython.core.display.Javascript object>

Uncomment and run the following cell to see the answer to this activity (after you try it!)

In [108]:
# %load ./solutions/W3/corpus_stats.py

<IPython.core.display.Javascript object>

### 3.C - Identification of end-of-life events  <a class="anchor" id="3-C"></a>
</br>
<center>
    <img src="./images/nb_3-1_flow_diagram-C.png"/>
</center>
</br>
Now that we have an intuition towards the fundamental concepts of natural language processing such that we can speak a common language, lets use what we've learnt to identify end-of-life events in maintenance work order short text. We treat end-of-life (EOL) events as any event that results in the inability of an item to function (e.g. functional failure). For example, "mechanical seal blown on pump" would indicate an EOL event for the pump.


There are numerous ways this can be tackled, but the two we will explore in this notebook include:
- 1. Basic keyword search (e.g. what we would do if we were using spreadsheet software), annd
- 2. Expanded keyword search by using word embeddings to identify trigger terms

For each of the proposed techniques that we will review, they become increasingly data intensive but more effective.
- Basic keyword search - no annotated data, least effective
- Expanded keyword search using embeddings - low amount of annotated data, more effective than basic keyword search

**⚡ Concept - Annotation**</br>
Annotation is the process of labeling data to guide models and teach them to predict the outcome we want. Consider the text `replace air compressor` - we may want a model that can predict the word `replace` is an `activity`, so we would annotate a number of examples texts expressing this information and hopefully be able to teach a model to perform this task automatically.

In [109]:
# Before we commence the following, we'll create a copy of the original data and normalise it with our cleaning function.
df_large_cleaned = df_large.copy()

# Lets pass the descriptions through our cleaner
df_large_cleaned["description"] = df_large_cleaned["description"].apply(
    lambda text: text_cleaner(text=text, norm_dict=word_normalisation_dictionary)
)

<IPython.core.display.Javascript object>

## 3.2.C.1 - Basic Keyword Search  <a class="anchor" id="3-C-1"></a>

The technique we'll explore is `basic keyword searching`, that is, like in spreadsheet software (Excel, etc.) we will search our data for words or phrases that may indicate an end-of-life event. For example, consider the texts:
- `replace` pump impeller
- Bredel pump `blown`
- `cracked` piston in piston pump
- PU001 `not pumping` efficiently

From these texts, we could elicit the terms `replace`, `blown`, `cracked`, `not pumping` as indicators of end-of-life. 

First, lets review our dataset and get some group intuitions for this task 

In [110]:
# Lets make a copy of our cleaned dataset
df_kws = df_large_cleaned.copy()

df_kws_len = len(df_kws)  # Get the length of the copied dataset to use for comparisons

<IPython.core.display.Javascript object>

In [111]:
print(f"Records in dataset: {len(df_kws)}")

Records in dataset: 45039


<IPython.core.display.Javascript object>

In [112]:
# Print dataset
df_kws.head(5)

,id,description,basic_start_date,priority,...,functional_loc_desc,functional_loc,area,object_desc
0,1,3y mec sdn replace conveyor belt cvr068,2/01/2017,High,...,Belt-CVR068,1071-30-25-09-CVR068-MECH-BLT001,mine,Belt
1,2,cvr029 replace conveyor belt,28/10/2020,High,...,Belt CVR029,1071-30-05-07-CVR029-MECH-BLT001,mine,Belt
2,3,144w mec sdn replace cvr069 conveyor belt,13/10/2018,High,...,Belt CVR069,1071-30-25-09-CVR069-MECH-BLT001,mine,Belt
3,4,3y mec sdn replace conveyor belt cvr069,5/05/2021,High,...,Belt CVR069,1071-30-25-09-CVR069-MECH-BLT001,mine,Belt
4,5,cvr029 replace conveyor belt,22/06/2018,High,...,Belt CVR029,1071-30-05-07-CVR029-MECH-BLT001,mine,Belt


<IPython.core.display.Javascript object>

### Activity 3.C.1 - Identifying end-of-life events (EOL) via key-word search  <a class="anchor" id="3-C-1-activity"></a>

Post your answers on [Menti]().

With reference to the work order dataset,
- What words or phrases would you use to search for end-of-life events?
- What limitations do you think there are to the approach of key-word search?

Lets use the `pandas` package to perform basic keyword search on our dataset.

This is how we could filter all of our records for the term `replace`, similar to what we saw in the previous weeks.

In [113]:
# Let's filter our pump dataset for all records that contain the term `replace`
# We will ignore the casing of the words here so that replace and Replace are treated equally.

search_term = "replace"  # Change to any EOL word/phrase

df_kws_replace = df_kws[df_kws["description"].str.contains(search_term, case=False)]
print(
    f"Number of records containing {search_term}: {len(df_kws_replace)}/{df_kws_len} ({len(df_kws_replace)/df_kws_len*100:0.2f}%)"
)

Number of records containing replace: 7352/45039 (16.32%)


<IPython.core.display.Javascript object>

We obviously know that there are numerous words and phrases that can indicate EOL, so lets expand our search.

In [114]:
# Lets expand our vocabulary and search for other words
# Note: the term `vocabulary` is used here similar to
# the dictionary definition, however the subject is specifically words that represent end-of-life events
eol_terms = ["replace", "change out"]
pattern = "|".join(eol_terms)

df_kws_eol = df_kws[df_kws["description"].str.contains(pattern, case=False)]
print(
    f"Number of records matching EOL dictionary: {len(df_kws_eol)}/{df_kws_len} ({len(df_kws_eol)/df_kws_len*100:0.2f}%)"
)

Number of records matching EOL dictionary: 7469/45039 (16.58%)


<IPython.core.display.Javascript object>

### 3.C.1 - Identifying end-of-life events (EOL) via key-word search
#### Take aways
- Key-word search is useful *IF* your data has no data quality issues *AND* you have an extensive vocabularly of search terms that have broad coverage of end-of-life terms.
- Key-word search does not help us distinguish whether the `item` in question is functional. For example, "replace pump paint" may include an EOL term "replace" but the semantics of the text is not something we would consider as evidence for statistical life data analysis.

#### Questions raised
- How do we find end-of-life terms to search for, outside of those most common such as `replace`, `change out`, etc?
- How do we deal with poor data quality that impacts our search? For example, `rpl` and `replc` being erroneous versions of `replace`.
- How do we isolate our searches for functional items that contribute to the lifetime of the asset rather than ancilliaries?

### 3.C.2 - Expanded keyword search using embeddings  <a class="anchor" id="3-C-2"></a>
The next technique we will explore uses the concept of word embeddings to create a dictionary (or "gazetteer") of search terms that we can use to expand our EOL terms. This is useful if we cannot exhaustively list all of the ways EOL events can be expressed in maintenance texts.

The initial steps we'll take here use the concepts we've already been exposed to, namely, ngrams and word embeddings.

In [115]:
# Lets make another copy of our cleaned dataset
df_expanded_kws = df_large_cleaned.copy()

<IPython.core.display.Javascript object>

In [116]:
# Lets create a corpus from our maintenance texts
mwo_corpus_expanded_kws = df_expanded_kws["description"].tolist()

print(mwo_corpus_expanded_kws[:3])

['3y mec sdn replace conveyor belt cvr068', 'cvr029 replace conveyor belt', '144w mec sdn replace cvr069 conveyor belt']


<IPython.core.display.Javascript object>

We can make the assumption that lowercasing the corpus will not impact the ability to identify end-of-life terms as these are not represented as proper nouns, etc. However, removing casing will improve our embeddings.

In [117]:
# Like we did in the fundamentals section, we will lowercase and tokenize all of our texts
tokenized_mwo_corpus_expanded_kws = [
    word_tokenize(text) for text in mwo_corpus_expanded_kws
]

print(tokenized_mwo_corpus_expanded_kws[:3])

[['3y', 'mec', 'sdn', 'replace', 'conveyor', 'belt', 'cvr068'], ['cvr029', 'replace', 'conveyor', 'belt'], ['144w', 'mec', 'sdn', 'replace', 'cvr069', 'conveyor', 'belt']]


<IPython.core.display.Javascript object>

In [118]:
# We know that ngrams capture important information, so we'll create a phraser model that automatically identifies ngrams from our corpus
phrase_model_expanded_kws = phrases.Phrases(
    tokenized_mwo_corpus_expanded_kws,
    min_count=5,
    threshold=10,
    connector_words=phrases.ENGLISH_CONNECTOR_WORDS,
)

<IPython.core.display.Javascript object>

In [120]:
# Lets check that our phraser model works as expected
phrase_model_expanded_kws[["change", "out", "converyor", "idler"]]

['change_out', 'converyor', 'idler']

<IPython.core.display.Javascript object>

In [119]:
# Here we'll train an embedding model on our phrased data
w2v_model_expanded_kws = Word2Vec(
    sentences=[phrase_model_expanded_kws[t] for t in tokenized_mwo_corpus_expanded_kws],
    vector_size=300,
    min_count=5,
    window=3,
    epochs=50,
)

<IPython.core.display.Javascript object>

Now that we have an embedding model trained on our corpus that has been passed through our phrasing model, we can exploit the numerical space of the embedding to find new EOL terms to improve the coverage of our keyword search process.

In [121]:
# Recall that given a word, we can find all of those that are similar.
# Hence, words used in similar contexts will share similar numerical values (be close in n-dimensional space)
# Therefore, we can start with a few seed terms and iteratively explore the embedding vector space to
# incrementally build up a vocabularly of EOL terms.

# Our initial kws terms (similar to the second part of the previous activity)
eol_expanded_kws_terms = ["replace", "change_out"]

<IPython.core.display.Javascript object>

In [122]:
def get_similar_terms(
    model, term: str, saved_terms: list, irrelevant_terms: list, topn: int
) -> tuple:
    """
    Function for finding terms similar to a provider term whilst discarding terms already seen.
    """

    tries = 0
    max_tries = 100
    similar_terms = []
    similar_terms_proba = []
    while len(similar_terms) <= topn:
        similar_terms_temp, similar_terms_proba_temp = zip(
            *model.wv.most_similar([term], topn=topn * 10)
        )

        # remove similar terms already in terms list
        new_terms_idx = [
            idx
            for idx, similar_term in enumerate(similar_terms_temp)
            if similar_term not in irrelevant_terms + saved_terms
        ]
        # slice terms and proba lists
        similar_terms_temp_slice = np.array(list(similar_terms_temp))[new_terms_idx]
        similar_terms_proba_temp_slice = np.array(list(similar_terms_proba_temp))[
            new_terms_idx
        ]

        similar_terms.extend(similar_terms_temp_slice)
        similar_terms_proba.extend(similar_terms_proba_temp_slice)

        tries += 1

        if tries == max_tries:
            return None, None

    return (similar_terms[:topn], similar_terms_proba[:topn])

<IPython.core.display.Javascript object>

In [124]:
## If you want to see what comes out of this function - uncomment this block of code.
# test_terms, test_probs = get_similar_terms(
#    model=cleaned_w2v_model,
#    term="replace",
#    saved_terms=["changed"],
#    irrelevant_terms=["replaced", "change_out"],
#    topn=10,
# )
# print(test_terms, test_probs)

<IPython.core.display.Javascript object>

As we can see, we can easily identify top-n phrases that are similar to some other given phrase. Now, if we knew some seed end-of-life phrases (such as 'replace', 'change out', etc.), we could easily search the embedding space to find all of the other ones using the semantics and syntactic properties represented in our learnt embedding space. An overview of the process we are going to perform is shown below.

<img src="./images/nb_3-1_term_expansion.png" width="75%"/>

In [125]:
# Application state
idx = 0
topn = 10
terms = ["replace", "change_out", "failed"]
irrelevant_terms = []
finished = False
sampled_terms, _ = get_similar_terms(
    model=cleaned_w2v_model,
    term=terms[0],
    saved_terms=terms,
    irrelevant_terms=irrelevant_terms,
    topn=topn,
)

# Application and logic
title = widgets.HTML(
    value=f"Term <b>{terms[0]}</b> (1/{len(terms)})",
)

button = widgets.Button(
    description="Save and continue",
    tooltip="Click to save and continue to next term",
    button_style="",
    icon="check",
)

finish_btn = widgets.Button(
    description="Im finished", tooltip="Click to finish", button_style="", icon="check"
)

multiselect = widgets.SelectMultiple(
    options=sampled_terms,
    value=[],
    rows=topn,
)
output = widgets.Output(layout={"border": "1px solid black"})


@output.capture()
def on_button_click(b):
    global idx, finished, saved_terms, terms, sampled_terms
    clear_output()
    finished = idx == len(terms) - 1

    # Add selected terms to running list
    terms.extend(list(multiselect.value))
    # Add terms not selected to irrelevant list
    irrelevant_terms.extend(list(set(sampled_terms) - set(multiselect.value)))

    print(f"Terms saved: {len(terms)}\nTerms Discarded: {len(irrelevant_terms)}")

    if not finished:
        idx += 1
        next_term = terms[idx]
        title.value = f"Term <b>{terms[idx]}</b> ({idx+1}/{len(terms)})"

        # Sample terms
        sampled_terms, sampled_probs = get_similar_terms(
            model=cleaned_w2v_model,
            term=next_term,
            saved_terms=terms,
            irrelevant_terms=irrelevant_terms,
            topn=topn,
        )

        multiselect.options = sampled_terms

    else:
        b.disabled = True
        multiselect.disabled = True


button.on_click(on_button_click)


@output.capture()
def on_finish_btn_click(b):
    global finished
    b.disabled = True
    multiselect.disabled = True
    button.disabled = True


finish_btn.on_click(on_finish_btn_click)

box_layout = widgets.Layout(
    display="flex", flex_flow="column", align_items="center", width="50%"
)

box = widgets.HBox(
    children=[title, multiselect, button, finish_btn, output], layout=box_layout
)

<IPython.core.display.Javascript object>

Here we are going to use a widget built using [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/index.html) to interactively identify end-of-life terms (phrases and words) to build up our end-of-life dictionary to improve coverage of our end-of-life identification.

In [126]:
# Show application
display(box)

<IPython.core.display.Javascript object>

In [128]:
# Lets save these to our working space (in case we need them at some later point)
with open("../data/eol_terms.txt", "w", encoding="utf-8") as outfile:
    for term in list(set(terms)):
        outfile.write(f"{term}\n")

<IPython.core.display.Javascript object>

Now that we've elicited our end-of-life terms and phrases, lets have a look a the terms we identified and those we disregarded.

In [129]:
# Lets look at the terms we identified
list(set(terms))[:10]

['need_replacing',
 'damage',
 'wont_reset',
 'collapsed',
 'rep',
 'unable_reset',
 'siezed',
 'replacing',
 'damaged',
 'trips']

<IPython.core.display.Javascript object>

In [130]:
# Lets look at the terms we discarded
irrelevant_terms[:10]

['cvr_131',
 'near',
 'cvr23',
 'status',
 'bco',
 'upgrade',
 'visual_inspection',
 'dislodged',
 'centre',
 '14']

<IPython.core.display.Javascript object>

Let's compare the coverage we got from using NLP techniques to find additional EOL terms.

In [131]:
# We need to remove the underscore added by our phrasing model before we can string search as these do not exist in the original texts
eol_expanded_kws_terms = [token.replace("_", " ") for token in list(set(terms))]

<IPython.core.display.Javascript object>

In [132]:
print(f"Number of expanded search terms: {len(eol_expanded_kws_terms)}")

Number of expanded search terms: 46


<IPython.core.display.Javascript object>

In [133]:
# As we can see, with only a small amount of effort we're able to identify many more records that may have potential for evidence in statistical data life analysis.

expanded_pattern = "|".join(eol_expanded_kws_terms)

df_expanded_kws_eol = df_expanded_kws[
    df_expanded_kws["description"].str.contains(expanded_pattern, case=False)
]

<IPython.core.display.Javascript object>

In [134]:
print(
    f"""
1. Basic keyword search
{len(df_kws_eol)}/{df_kws_len} ({len(df_kws_eol)/df_kws_len*100:0.2f}%)

2. Expanded keyword search
{len(df_expanded_kws_eol)}/{df_kws_len} ({len(df_expanded_kws_eol)/df_kws_len*100:0.2f}%)

Additional records identified
{len(df_expanded_kws_eol)-len(df_kws_eol)} ({(1 - (len(df_kws_eol)/len(df_expanded_kws_eol))) * 100:0.1f}%)
"""
)


1. Basic keyword search
7469/45039 (16.58%)

2. Expanded keyword search
11371/45039 (25.25%)

Additional records identified
3902 (34.3%)



<IPython.core.display.Javascript object>

### Activity 3.C.2 - Expanded keyword search using embeddings  <a class="anchor" id="3-C-2-activity"></a>

Post your answers in [Menti]()
- How many terms/phrases did you find by searching the embedding space?
- What are your thoughts on the process we've employed so far?
- What limitations do you think there are to this approach?

### 3.D - Classification of end-of-life events  <a class="anchor" id="3-D"></a>
<img src="./images/nb_3-1_flow_diagram-D.png"/>

Now that we have an idea of how to identify end-of-life events in maintenance texts, we need to know how to classify these pieces of evidence as either failure or suspension before we can perform Weibull analysis.


- End-of-life classification: 
Now that we have an idea of the process of eliciting end-of-life events from unstructured text in maintenance work orders, how do we classify these into failures or suspensions? 

- Primer on end-of-life identification and classification: 
Given a set of MWOs, how do we determine which mention end-of-life events, and of those that do, how do we determine whether they are failures or suspensions?

In [135]:
# Lets make another copy of the cleaned dataframe
df_eol_clf = df_large_cleaned.copy()

<IPython.core.display.Javascript object>

In [136]:
# Lets use the EOL terms we extracted to mark each row in the dataframe as either having an EOL or not
# Recall that the `expanded_pattern` was created in the previous section

df_eol_clf["eol"] = df_eol_clf["description"].str.contains(expanded_pattern, case=False)

<IPython.core.display.Javascript object>

In [137]:
# Lets look at the boolean column we added with the descriptions and order types
df_eol_clf[["description", "eol", "wo_order_type"]].iloc[:25]

,description,eol,wo_order_type
0,3y mec sdn replace conveyor belt cvr068,True,PM02
1,cvr029 replace conveyor belt,True,PM01
2,144w mec sdn replace cvr069 conveyor belt,True,PM02
3,3y mec sdn replace conveyor belt cvr069,True,PM02
4,cvr029 replace conveyor belt,True,PM01
5,cvr062 change belt,True,PM01
6,cvr030 eng project works,False,PM01
7,2y mec sdn replace conveyor belt cvr323,True,PM02
8,cb mec sdn replace conveyor belt cvr061,True,PM02
9,cvr064 belt replacement,True,PM01


<IPython.core.display.Javascript object>

Now that we have identified records with potential EOL events, we need to classify them as either failure or suspension. 

To classify an identified end-of-life event as failure or suspension, we'll leverage the fortuitous data fields accompanying the work order texts.

For our dataset, we'll use the PM codes where we'll use 'expert logic' to say that anything that is PM01/PM03 (corrective/breakdown - if PM03 is applicable) is a failure and anything that is preventative/planned (PM02) is a suspension. If the data provides more granular information, you could use that instead. At the minimum, maintenance work orders should contain PM codes.

In [138]:
# Here we will specify the column we'll use to classify the work as failure/suspension
fail_suspension_col = "wo_order_type"

<IPython.core.display.Javascript object>

In [139]:
# Lets take a look at all the unique values in this column
# Like we saw earlier, we can see a clear distribution of the types of work orders that are created.
df_eol_clf[fail_suspension_col].value_counts()

PM02    25640
PM01    19384
PM05        9
PM03        6
Name: wo_order_type, dtype: int64

<IPython.core.display.Javascript object>

Using the values in the failure/suspension column, we can perform the classification of the EOL events.

In [140]:
# Here we will say that anything that is corrective/breakdown is PM01 and
# anything that is planned/preventative is PM02. If the order type is anything else (e.g. PM03/PM05) we'll ignore it
fail_values = ["PM01"]
suspension_values = ["PM02"]

<IPython.core.display.Javascript object>

In [141]:
# Lets add a new column to our dataset that classifies the record as a failure (F) or suspension (S)
df_eol_clf["fs_clf"] = np.where(
    (df_eol_clf[fail_suspension_col].isin(suspension_values)),
    "suspension",
    np.where(df_eol_clf[fail_suspension_col].isin(fail_values), "failure", "other"),
)

<IPython.core.display.Javascript object>

In [142]:
# Lets take a look at the work orders classified as 'other' - do we need to include these?
df_eol_clf[df_eol_clf["fs_clf"] == "other"]

,id,description,basic_start_date,priority,...,area,object_desc,eol,fs_clf
56,57,cvr030 belt replacement breakdown,7/06/2018,High,...,mine,Fixed Stacker Boom Conveyor,True,other
334,335,cvr068 minprovise recovery team,4/03/2020,High,...,mine,Stockyard Reclaim Conveyor,False,other
1335,1336,cvr068 eilbeck labour hire,3/03/2020,High,...,mine,Gravity Take Up,False,other
1458,1459,jaw crusher integration,27/02/2017,High,...,mine,Overland Conveyor South,False,other
3269,3270,supply spare rollers sct,7/04/2016,Immediate,...,mine,Idlers,False,other
5800,5801,supply 5 carry rollers sct,5/03/2016,Urgent,...,mine,Idlers,False,other
27253,27254,spares wo cvr030 return frames,4/10/2019,High,...,mine,Idlers,False,other
27254,27255,supply spare gearbox sct,10/12/2015,Urgent,...,mine,Gearbox,False,other
27447,27448,sct request spare - 1250kw motor,20/04/2016,Medium,...,port,Motor,False,other
27448,27449,replace drive 2 motor,1/10/2016,Immediate,...,port,Drive Assembly,True,other


<IPython.core.display.Javascript object>

Now that we have both identified potential EOL events and provided a classification for them based on a structured fortuitous field, we can now filter the dataset for records that may be used as evidence for statistical data life analysis. Here, we will remove all records that do not have an EOL event identified.

In [143]:
# By default, pandas will return the slice of the dataframe for all the instances that are True
df_eol_filtered = df_eol_clf[df_eol_clf["eol"]]

<IPython.core.display.Javascript object>

In [144]:
print(f"Records with potential EOL events: {len(df_eol_filtered)}/{len(df_large)}")

Records with potential EOL events: 11371/45039


<IPython.core.display.Javascript object>

Before we can move onto using these records in our analysis for reliability measures, we first need to try and reason about whether the events actually manifested. For example, if a work order text indicated the change out of a component but it either hasn't been actioned (has no start date) or has no, or low, associated cost/time, do we believe that this should be included in our analysis?

### Activity 3.D - Classification of end-of-life events  <a class="anchor" id="3-D-activity"></a>

Post your answers to [Menti]()
- What fields would you use to help reason about whether an end-of-life event actually occurred?

### 3.2.E - Reasoning about end-of-life events  <a class="anchor" id="3-E"></a>
<img src="./images/nb_3-1_flow_diagram-E.png"/>

Using the dataframe with identified EOL events and classifications, we can further filter the dataframe using intuition. This approach is iterative, but gives us a good rule of thumb.

First, we know that we need the time between event to calculate reliability measures, so lets remove all records that do not have an actual start date.

In [145]:
# Lets perform a preliminary filter of the dataframe.
# We know to get reliability measures such as MTTF/MTBF we need to know a date, for this work we'll take the actual start date as being the point of end-of-life
# However, this date could also be the creation date of the work order or associated notification.
df_eol_filtered = df_eol_filtered[~df_eol_filtered["actual_start_date"].isna()]

<IPython.core.display.Javascript object>

In [146]:
print(f"Number of filtered records with actual start date {len(df_eol_filtered)}")

Number of filtered records with actual start date 8490


<IPython.core.display.Javascript object>

Second, we know that from experience that just because a maintenance work order is raised, it doesn't mean it is executed, so we need a proxy for determining whether work was carried out. Here, we'll use the fortuitious field of actual total cost. However, fields from other linked data sources could also be used, such as spares information to determine whether a part was changed out.

Lets analyse the data we currently have to gauge what threshold seems reasonable to set initially. Note we could use our profiled data at the start of this notebook to gain similar insight.

In [147]:
# We can use the handly describe function to quickly view the stats on our numerical columns
df_eol_filtered.describe()

,id,total_actual_costs
count,8490,8490
mean,13991.3,11560.8
std,12229.6,63451.5
min,1,-33967.6
25%,3634.5,318.84
50%,9153.5,1003.16
75%,27837.5,4560.36
max,45055,3.51425e+06


<IPython.core.display.Javascript object>

Lets set a general threshold for all of the assets at once, however in the future you could improve this logic by finding asset specific thresholds.

In [148]:
cost_threshold = 2000  # dollars

<IPython.core.display.Javascript object>

In [149]:
## Example process for finding mean values for total acutal cost for each floc in our data - We'll leave this as an exercise for those interested to extend this notebook.
# df_floc_groups = df_eol_filtered.groupby(by=['functional_loc'])

## Here we are going to find the mean value for total actual cost as an initial threshold for our model
# floc_thresholds = {}
# for name, group in df_floc_groups:
#    print(name, group)
#
#    floc_thresholds[name] = {'total_actual_costs': group['total_actual_costs'].mean()}

<IPython.core.display.Javascript object>

In [150]:
df_eol_filtered = df_eol_filtered[
    (cost_threshold <= df_eol_filtered["total_actual_costs"])
]

<IPython.core.display.Javascript object>

In [151]:
print(f"Number of work orders after filtering operations: {len(df_eol_filtered)}")

Number of work orders after filtering operations: 3182


<IPython.core.display.Javascript object>

Using the two expert thresholds on the structured fortuitious data fields, we can move onto converting our classified records into a format suitable for Weibull analysis. However, it should be noted that the thresholds we've blanket applied can be different for each asset. Therefore, eliciting more accurate representations of cost/work requirements for different assets will improve the classification process.

### Activity 3.E - Reasoning about end-of-life events  <a class="anchor" id="3-E-activity"></a>
</br>

Post your answers in [Menti]()
- What are the limitations to this approach? How could we make it better?

### 3.F - Statistical data life analysis  <a class="anchor" id="3-F"></a>
<img src="./images/nb_3-1_flow_diagram-F.png"/>

Now that we have filtered our initial dataset down to records that are likely to have had an end-of-life event manifest, we can convert our data into the format expected by the Reliability package for Weibull analysis.

Before we do this, lets first set a threshold for the minimum number of evidence we expect an asset to have before it can have an analysis performed on it. Typically, a minimum of 5 points of evidence are required for statistical significance. By setting it here, we have clear control of the process.

In [152]:
min_evidence_points = 5

<IPython.core.display.Javascript object>

In [153]:
# First lets check out how many failures/suspensions we have in our dataset
df_eol_filtered["fs_clf"].value_counts()

failure       1858
suspension    1323
other            1
Name: fs_clf, dtype: int64

<IPython.core.display.Javascript object>

This next step will group each of the records by their functional location and create a failure/suspension dataset.

In [154]:
groups = df_eol_filtered.groupby(["functional_loc"])

<IPython.core.display.Javascript object>

In [155]:
print(f"Number of functional locations at the start of analysis: {len(groups)}")

Number of functional locations at the start of analysis: 530


<IPython.core.display.Javascript object>

In [156]:
# Here we will iterate over the groups of functional locations,
# skipping past any that do not meet the minimum evidence requirements
# Feel free to uncomment some of the print statements to get a better understanding of what is happening

fs_data_per_group = {}
for name, values in groups:

    if len(values) < min_evidence_points:
        #         print('N\t', name)
        continue
    else:
        #         print(len(values))
        #         print('Y\t', name)
        values = values[["actual_start_date", "fs_clf"]]

        #         print(values.head())

        # Lets sort the groups data by actual_start_date so that the earliest come first
        fs_data = values.sort_values(by=["actual_start_date"], ascending=True)

        #         print(fs_data.head())

        # Calculate the time between event (we'll use days here)
        fs_data["time"] = fs_data["actual_start_date"].diff() / np.timedelta64(1, "D")
        fs_data["time"] = fs_data["time"].fillna(0)

        #         print(fs_data)

        # Encode failures and suspensions as 1 and 0, respectively
        fs_data["fs_clf"].replace(to_replace="failure", value=1, inplace=True)
        fs_data["fs_clf"].replace(to_replace="suspension", value=0, inplace=True)

        #         print(fs_data)

        fs_data_per_group[name] = fs_data[["fs_clf", "time"]]

<IPython.core.display.Javascript object>

In [157]:
# How many flocs do we have sufficient evidence for?
print(
    f"Number of functioal locations with sufficient evidence: {len(fs_data_per_group)} / {len(groups)}"
)

Number of functioal locations with sufficient evidence: 177 / 530


<IPython.core.display.Javascript object>

For each of the groups that have sufficient evidence, we can now use the Reliability package to compute their mean time values using 2-parameter Weibull analysis programmatically.

In [158]:
# Uncomment the print statements to get a better understanding of what is happening.

results_per_group = {}
for name, fs_data in fs_data_per_group.items():

    fs_data = fs_data[
        0 < fs_data["time"]
    ]  # Do not take into account any events that have 0 time

    failures = fs_data[fs_data["fs_clf"] == 1]
    right_censored = fs_data[fs_data["fs_clf"] == 0]  # suspensions

    failure_times = failures["time"].tolist()
    right_censored_times = right_censored["time"].tolist()

    #     print(failures)
    #     print(right_censored)

    if 1 < len(failures):
        wbfit = Fit_Weibull_2P(
            failures=failure_times,
            right_censored=right_censored_times
            if len(right_censored_times) > 0
            else None,
            show_probability_plot=False,
            print_results=False,
        )

        mean = wbfit.distribution.mean

        if 365 * 10 < mean:
            print(f"{name} requires review - very large mean ({mean:0.0f} units)")
        else:
            results_per_group[name] = {
                "alpha": wbfit.alpha,
                "beta": wbfit.beta,
                "mean": mean,
                "time_on_test": fs_data["time"].sum(),
                "evidence": len(fs_data),
            }
    else:
        print(f"{name} only has censored events")
        continue

1071-30-05-01-CVR102-STRC-CHU001 only has censored events
1071-30-05-01-CVR123-MECH-SCP001 only has censored events
1071-30-05-02-CVR223-MECH-SCP001 only has censored events
1071-30-05-03-CVR302-STRC-CHU002 only has censored events
1071-30-05-05-CVR025-MECH-SCP001 only has censored events
1071-30-05-07-CVR030-MECH-SCP001 only has censored events
1071-30-10-04-CVR431-MECH-BLT001 only has censored events
1071-30-15-00-CVR041-MECH-BLT001 requires review - very large mean (4789 units)
1071-30-15-00-CVR041-MECH-SCP001 only has censored events
1071-30-15-00-CVR042-MECH-BLT001 only has censored events
1071-30-15-00-CVR042-MECH-SCP001 only has censored events
1071-30-25-01-CVR056-MECH-IDL001 only has censored events
1071-30-25-01-CVR061-MECH-SCP001 only has censored events
1071-30-25-01-CVR062-MECH-SCP001 only has censored events
1071-30-25-01-CVR063-MECH-BLT001 only has censored events
1071-30-25-01-CVR063-MECH-SCP001 only has censored events
1071-30-25-01-CVR064-MECH-SCP001 only has censored

<IPython.core.display.Javascript object>

In [159]:
# Lets round our values to make them nicer to work with and view what we've found
df_results = pd.DataFrame.from_dict(results_per_group).T
df_results = df_results.round({"alpha": 0, "beta": 1, "mean": 0})
df_results.head()

,alpha,beta,mean,time_on_test,evidence
1071-30-05-01-CVR101-MECH-IDL001,285,2.2,253,1543,8
1071-30-05-01-CVR102,117,0.8,138,1011,9
1071-30-05-01-CVR102-ELEC-MDT001,267,0.8,317,1282,4
1071-30-05-01-CVR102-MECH-BLT001,484,0.8,569,1488,4
1071-30-05-01-CVR102-MECH-IDL001,149,1.6,134,1940,20


<IPython.core.display.Javascript object>

Lets join the object descriptions back onto our data using a `left join`, this will allow us to categorise our data when we're visualising our results.

In [160]:
# First we'll pop out the index and make it a column (we'll use this as a key to join on)
df_results = df_results.rename_axis("functional_loc").reset_index()

<IPython.core.display.Javascript object>

In [161]:
# Now we'll add the object_desc column to the results dataframe joining on the functional_loc column
df_results_with_objs = pd.merge(
    df_results,
    df_large[["functional_loc", "object_desc"]].drop_duplicates(),
    on="functional_loc",
    how="left",
)

<IPython.core.display.Javascript object>

In [162]:
df_results_with_objs.head(5)

,functional_loc,alpha,beta,mean,time_on_test,evidence,object_desc
0,1071-30-05-01-CVR101-MECH-IDL001,285,2.2,253,1543,8,Idlers
1,1071-30-05-01-CVR102,117,0.8,138,1011,9,Secondary Sizer Feed
2,1071-30-05-01-CVR102-ELEC-MDT001,267,0.8,317,1282,4,Metal Detector
3,1071-30-05-01-CVR102-MECH-BLT001,484,0.8,569,1488,4,Belt
4,1071-30-05-01-CVR102-MECH-IDL001,149,1.6,134,1940,20,Idlers


<IPython.core.display.Javascript object>

In [163]:
fig_bubble = px.scatter(
    df_results_with_objs,
    x="alpha",
    y="beta",
    size="evidence",
    color="mean",
    symbol="object_desc",
    hover_name=df_results.index,
    color_continuous_scale="Bluered_r",
)
fig_bubble = fig_bubble.update_layout(
    title_text=f"Overview of Results ({len(df_results)} assets)"
)
fig_bubble = fig_bubble.update_layout(coloraxis_colorbar=dict(orientation="h"))

<IPython.core.display.Javascript object>

Tips: 
- Double click legend items to isolate them
- Drag your mouse cursor to zoom into regions and navigate

In [165]:
fig_bubble.show()

<IPython.core.display.Javascript object>

The approach we've explored has allowed us to quickly identify, classify and reason about EOL events exhibited in maintenance work order records. Using this information, we have been able to extract reliability measures in a standardised, and scalable, manner. However, the approach we've demonstrated is not a panacea, but rather should be used as a rule of thumb and a method for quickly structuring data for this task. In the next notebook, we will dive into this further.

## Summary of Data-Driven Reliability Metrics (Part 1)  <a class="anchor" id="summary"></a>

We've learnt a lot of new concepts in this notebook such as exploratory data analysis, fundamentals of NLP, how to clean text data, and how to scale our Weibull analysis over thousands of work orders. In the next notebook (*W3-2 Data-Driven Reliability Metrics - Analysis*) we are going to use what we have learnt in this notebook to analyse our results using interactive visualisation techniques similar to what you might find in BI tools. 

# Appendix  <a class="anchor" id="appendix"></a>

**⚡ Concept - Stemming and lemmatization**</br>
Imagine you have the following texts `replaced pump - working well` and `replace pump - working good`. It may be desirable to reduce each word to a root form to reduce the size of the vocabulary within our corpus. Stemming is a technique that stems all words to a common root by removing or replacing word suffixes (e.g. "replaced" to "replace") whereas lemmatization returns the base form of words (e.g. "well" to "better")

In [166]:
# For this section we'll need access to an external resource called WordNet
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\61437\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\61437\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

<IPython.core.display.Javascript object>

In [167]:
# Lets instantiate a stemmer from NLTK
stemmer = PorterStemmer()

<IPython.core.display.Javascript object>

In [168]:
# Try 'replace pump - not pumping'
sentence_to_stem = "replace pump - not pumping"
sentence_to_stem_tokenized = word_tokenize(sentence_to_stem)

# Like the other exercises, we need to tokenize our sentence
stemmed_sentence = [stemmer.stem(token) for token in sentence_to_stem_tokenized]
print(stemmed_sentence)

['replac', 'pump', '-', 'not', 'pump']


<IPython.core.display.Javascript object>

In [169]:
# Lets instantiate a lemmatizer from NLTK
lemmatizer = WordNetLemmatizer()

<IPython.core.display.Javascript object>

In [170]:
# interchange, interchanger, interchanging

sentence_to_lemma = "interchange interchanger"
sentence_to_lemma_tokenized = word_tokenize(sentence_to_lemma)

lemmatized_sentence = [
    lemmatizer.lemmatize(token) for token in sentence_to_lemma_tokenized
]
print(lemmatized_sentence)

['interchange', 'interchanger']


<IPython.core.display.Javascript object>

**⚡ Concept - Concordancing**</br>
Concordancing gives us a view of every occurrence of a given word, together with some context. For example, we may be interested in the context that the word `seized` occurs in which may result in `...bearing seized on pulley...` and `...lack of lube - seized ...`.

Being able to analyse the use of words in context allows us to quicky get an understanding of what is happening in our texts.

In [171]:
# Before we can perform concordancing, we need to convert our MWO corpus into a single string and then
# into a special NLTK text object. We'll need to put a special token into the object to delineate between texts.
special_token = " | "
texts_for_concordancing = Text(word_tokenize(f"{special_token}".join(mwo_corpus)))

<IPython.core.display.Javascript object>

Lets take a look at concordances for single words such as `broken`, or negation such as `not`.

In [172]:
texts_for_concordancing.concordance("not")

Displaying 25 of 284 matches:
ate & Inse > Inse | Outstanding PO DO NOT TECO | CVR123 Replace Impact Plate & 
 Idle | CVR063 REPL Lower Insert | Do Not TECO outstanding PO | 24W MEC SDN REP
TRIAL - CVR030 Impact Idlers | # # DO NOT TECO # # CVR241 Hot Splice Repair | 8
ing on overload . | Outstanding PO Do Not TECO | 24W MEC SDN REPL H/Chute Liner
rd & Soft Skirts | Reported load cell not communicating | CVR025 Replace Return
lers | CVR041 Replace GTU Rubber | DO NOT TECO , WAITING FOR PARTS , CVR223 M |
t | Preps - Conveyor Belt CVR123 | DO NOT TECO CVR062-Check brake fail to a | R
ace skirts and clean | CVR042 U/Speed Not Working | CB MEC SDN RPL Brake Pads D
Power Cell - Siemens | METAL DETECTOR NOT RESETTING | CVR123-CVR023 Repair Tran
CVR123 - replace damaged rollers | DO NOT TECO - IDLER FRAMES ON ORDER | Replac
CVR029 & CVR030 | RTD 0211 Drive side not reading | CVR341 Refurbish Secondary 
lty multi I\O card | CVR029 Sirens Do Not Sound on Start Up | SSC056-CVR001 TLO
DN REPL Im

<IPython.core.display.Javascript object>

We can also look at concordances between multiple words, such as "not working"

In [173]:
texts_for_concordancing.concordance(["not", "working"])

Displaying 25 of 66 matches:
skirts and clean | CVR042 U/Speed Not Working | CB MEC SDN RPL Brake Pads DRV1 
e Failed Idlers | Mulitple lights not working on CVR029 | CVR030 Replace DRV002
42 Weightometer Alignment | Brake not working | Relace 8 x Tail End Return Roll
ed V Worn Frame | DBD002 Lighting not working in Auto | CVR070 Fit Conduits to 
 Metal Detector | cvr030 lighting not working | Isolating and suspending for op
g repairs on CVR025 | Tilt switch not working | replace return rollers | CVR065
d Frame | CVR023 beacon HRN152 is not working | Scrubbing Line 5 BLS5118 U\S | 
sfer Chute CVR223 | CVR223 lights not working | CVR068 Poly Return Roller Upgra
t switches | CVR023 Tunnel lights not working | CVR023 Tunnel emergency lightin
Pizza Cutters | CVR123 thermostat not working . | CVR431 BDS 4134 Broken off | 
 ZSL3148 fa | CVR302 weightometer not working | Weighto no Speed reference CVR1
ft skirt | CVR030 Load Cell XT001 not working | CVR061 Inspect pullwire cotton 
56 | CVR023

<IPython.core.display.Javascript object>

Due to the terse nature of maintenance work order short text, concordancing only provides a glimpse into the context that word/phrases are occurring. However, if you have longer documents such as long text or reports, this can provide quick insight.